Takes in .AWD, sleep log if it exists, calendar, and spits out some pretty pictures

In [1]:
import os,sys,inspect
current_dir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parent_dir = os.path.dirname(current_dir)
sys.path.insert(0, '/data/MoodGroup/actigraphy/gavi/actigraPy') 

import actigraPy.actigraPy as act
import importlib
import numpy as np
import pandas as pd
from datetime import datetime
from datetime import timedelta

/data/MoodGroup/actigraphy/gavi/actigraPy/actigraPy/actigraPy.py:23: MatplotlibDeprecationWarning: 
The matplotlib.backends.tkagg module was deprecated in Matplotlib 3.0 and will be removed in 3.2.
  import matplotlib.backends.tkagg as tkagg


If you want to run the test data, run this cell instead of the one below

In [2]:
# data and log directories
sub = 'TEST' #subject NUMBER as STRING
sub_long = 'TEST'

out_dir = '/data/MoodGroup/actigraphy/gavi/actigraPy/data/output' #output directory
data_dir = '/data/MoodGroup/actigraphy/gavi/actigraPy/data'

if not os.path.exists(out_dir):
    os.mkdir(out_dir)


## Subject and directory info

In [3]:
fn = {}
#.AWD
awd_fn = os.path.join(data_dir, '%s.AWD'%sub_long)
#sleeplog
fn['cal']= os.path.join(data_dir, '%s_calendar_log.xls'%sub_long)
#calendar
fn['log'] = os.path.join(data_dir, '%s_sleeplog.xls'%sub_long)
logs = []

if os.path.isfile(awd_fn):
    for name in fn.keys():
        if os.path.isfile(fn[name]):
            print("%s has %s file at %s"%(sub,name,fn[name]))
            logs.append(fn[name])
        else:
            print("no " + name)
            fn[name] = ''
else:
    print("DO NOT CONTINUE!! THERE IS NO AWD FILE")

TEST has cal file at /data/MoodGroup/actigraphy/gavi/actigraPy/data/TEST_calendar_log.xls
TEST has log file at /data/MoodGroup/actigraphy/gavi/actigraPy/data/TEST_sleeplog.xls


### Make Mtimes file

Figure out where to clip awd file

In [4]:
awd_dat = act.read_AWD(awd_fn)


In [5]:
#read AWD file
awd_dat = act.read_AWD(awd_fn)
idx={'start':[0],'end':[len(awd_dat['dt_list'])-1]}
#get the start and stops from each log if they exist
for log in logs:
    print(log)
    log_dat, kw_dat, comments = act.read_log(log,awd_dat)
    
    if 'watch_on' in kw_dat.keys():
        on_date = kw_dat['watch_on'].iloc[0]['OnDate']
        on_time = kw_dat['watch_on'].iloc[0]['OnTime']
        on = datetime(on_date.year,on_date.month,on_date.day,on_time.hour,on_time.minute)
        #check if that time is in dt_list:
        if awd_dat['dt_list'].count(on) > 0:
            on_idx=awd_dat['dt_list'].index(on)
            idx['start'].append(on_idx)

    if 'watch_off' in kw_dat.keys():
        off_date = kw_dat['watch_off'].iloc[0]['OffDate']
        off_time = kw_dat['watch_off'].iloc[0]['OffTime']
        off = datetime(off_date.year,off_date.month,off_date.day,off_time.hour,off_time.minute)
        if awd_dat['dt_list'].count(off) > 0:
            off_idx=awd_dat['dt_list'].index(off)
            idx['end'].append(off_idx)
 
idx

/data/MoodGroup/actigraphy/gavi/actigraPy/data/TEST_calendar_log.xls
     OffDate   OffTime     OnDate    OnTime                Comment
0        NaT       NaN 2016-02-08  10:00:00                  start
1 2016-02-07  13:00:00 2016-02-07  14:30:00  Fake, for bug testing
2 2016-02-08  13:00:00 2016-02-08  14:30:00                     3T
3 2016-02-08  21:30:00 2016-02-09  07:00:00            Sleep study
4 2016-02-09  14:00:00 2016-02-09  16:00:00                     7T
5 2016-02-10  10:00:00 2016-02-10  12:00:00                    NaN
6 2016-02-10  21:30:00 2016-02-11  07:00:00            Sleep study
7 2016-02-11  09:00:00 2016-02-11  09:40:00               Infusion
8 2016-02-11  15:00:00 2016-02-11  17:00:00                    MEG
[(0, 74, '3T'), (494, 1064, 'Sleep study'), (1484, 1604, '7T'), (2684, 2804, ''), (3374, 3944, 'Sleep study'), (4064, 4104, 'Infusion'), (4424, 4544, 'MEG')]
/data/MoodGroup/actigraphy/gavi/actigraPy/data/TEST_sleeplog.xls
     OffDate   OffTime     OnDate    O

{'start': [0, 144], 'end': [4741]}

In [6]:
start = max(idx['start'])
end = min(idx['end'])
print('start = %d, end = %d'%(start,end))

start = 144, end = 4741


## <font color='red'>Make sure the starts and ends make sense before clipping the data</font>


In [33]:
#modify these according to output from above!!
lim = [awd_dat['DateTime'][start],awd_dat['DateTime'][end]]
clip_dat = act.clip_dat(lim,awd_dat)

Now just run these cells in order to extract comments and write Mtimes

Testing/playing w code

In [29]:
importlib.reload(act)
log_dat,kw_dat = act.read_log(fn['cal'],awd_dat,oldvsn=False)


     OffDate   OffTime     OnDate    OnTime                Comment
0        NaT       NaN 2016-02-08  10:00:00                  start
1 2016-02-07  13:00:00 2016-02-07  14:30:00  Fake, for bug testing
2 2016-02-08  13:00:00 2016-02-08  14:30:00                     3T
3 2016-02-08  21:30:00 2016-02-09  07:00:00            Sleep study
4 2016-02-09  14:00:00 2016-02-09  16:00:00                     7T
5 2016-02-10  10:00:00 2016-02-10  12:00:00                    NaN
6 2016-02-10  21:30:00 2016-02-11  07:00:00            Sleep study
7 2016-02-11  09:00:00 2016-02-11  09:40:00               Infusion
8 2016-02-11  15:00:00 2016-02-11  17:00:00                    MEG


In [32]:
log_dat['idx']

[(0, 74, '3T'),
 (494, 1064, 'Sleep study'),
 (1484, 1604, '7T'),
 (2684, 2804, ''),
 (3374, 3944, 'Sleep study'),
 (4064, 4104, 'Infusion'),
 (4424, 4544, 'MEG')]

Real notebook continues here

In [71]:
importlib.reload(act)
#Make master mk_idx
mk_idx=act.get_markers(awd_dat)

In [72]:
mk_idx['z']

[(0, 79, ''),
 (87, 135, ''),
 (153, 165, ''),
 (215, 227, ''),
 (319, 347, ''),
 (351, 389, ''),
 (529, 547, ''),
 (553, 569, ''),
 (691, 729, ''),
 (751, 1159, ''),
 (1197, 1211, ''),
 (1293, 1313, ''),
 (1319, 1339, ''),
 (1473, 1641, ''),
 (1879, 1905, ''),
 (2055, 2091, ''),
 (2127, 2155, ''),
 (2161, 2551, ''),
 (2557, 2593, ''),
 (2599, 2651, ''),
 (2723, 2789, ''),
 (2991, 3007, ''),
 (3029, 3043, ''),
 (3051, 3093, ''),
 (3241, 3293, ''),
 (3301, 3321, ''),
 (3381, 3393, ''),
 (3401, 3413, ''),
 (3471, 3493, ''),
 (3607, 3935, ''),
 (3941, 3955, ''),
 (4005, 4039, ''),
 (4047, 4063, ''),
 (4069, 4083, ''),
 (4095, 4177, ''),
 (4239, 4253, ''),
 (4283, 4301, ''),
 (4381, 4407, ''),
 (4425, 4509, ''),
 (4679, 4693, ''),
 (4715, 4727, '')]

[(143, 689, ''), (1472, 1645, ''), (2094, 3237, '')]

In [73]:
importlib.reload(act)
#Read logs
for name in fn.keys():
    if os.path.isfile(fn[name]):
        log_dat,kw_dat= act.read_log(fn[name],awd_dat,oldvsn=False)
        mk_idx[name]=log_dat['idx']
        


     OffDate   OffTime     OnDate    OnTime                Comment
0        NaT       NaN 2016-02-08  10:00:00                  start
1 2016-02-07  13:00:00 2016-02-07  14:30:00  Fake, for bug testing
2 2016-02-08  13:00:00 2016-02-08  14:30:00                     3T
3 2016-02-08  21:30:00 2016-02-09  07:00:00            Sleep study
4 2016-02-09  14:00:00 2016-02-09  16:00:00                     7T
5 2016-02-10  10:00:00 2016-02-10  12:00:00                    NaN
6 2016-02-10  21:30:00 2016-02-11  07:00:00            Sleep study
7 2016-02-11  09:00:00 2016-02-11  09:40:00               Infusion
8 2016-02-11  15:00:00 2016-02-11  17:00:00                    MEG
     OffDate   OffTime     OnDate    OnTime Comment
0        NaT       NaN 2016-02-08  15:40:00   start
1 2016-02-09  00:42:00 2016-02-09  01:20:00  shower
2 2016-02-09  13:45:00 2016-02-09  16:35:00  7T MRI
3 2016-02-09  23:25:00 2016-02-10  00:05:00  shower
4 2016-02-10  19:15:00 2016-02-10  20:07:00  shower
5 2016-02-11  15:0

In [99]:
importlib.reload(act)
act.write_Mtimes(awd_dat,mk_idx,os.path.join(out_dir,sub_long))

,OnDate,OnTime,OffDate,OffTime,marker,Comment
0,08-Feb-16,01:16 PM,08-Feb-16,02:30 PM,cal,3T
1,08-Feb-16,01:16 PM,08-Feb-16,02:35 PM,z,
2,08-Feb-16,02:43 PM,08-Feb-16,03:31 PM,z,
3,08-Feb-16,03:49 PM,08-Feb-16,04:01 PM,z,
4,08-Feb-16,04:51 PM,08-Feb-16,05:03 PM,z,
5,08-Feb-16,06:35 PM,08-Feb-16,07:03 PM,z,
6,08-Feb-16,07:07 PM,08-Feb-16,07:45 PM,z,
7,08-Feb-16,09:30 PM,09-Feb-16,07:00 AM,cal,Sleep study
8,08-Feb-16,10:05 PM,08-Feb-16,10:23 PM,z,
9,08-Feb-16,10:29 PM,08-Feb-16,10:45 PM,z,


## Make graphs

In [ ]:
plots = {}
del_com={}
for name in fn.keys():
    if name in mk_idx.keys():
        plots[name]=mk_idx[name] - start
        del_com[name]=((np.where(plots[name]<0)[0])//2).tolist()[::2]
        plots[name]=plots[name][np.where(plots[name]>0)]

In [ ]:
coms=[]
idxs=[]
for mm in mc_dict.keys():
    print(mm)
    idxs= idxs +(mc_dict[mm]['idxs'])
    coms= coms + (mc_dict[mm]['comments'])
    if mm in del_com.keys():
        for i in range(0,len(del_com[mm])):
            idxs.pop(0)
            coms.pop(0)
zipped_pairs = zip(idxs,coms)
gc_coms = [x for _, x in sorted(zipped_pairs)]
gc_idx = np.sort(idxs) - start

gc=[np.asarray(gc_idx),gc_coms]

In [ ]:
plots['cal']

In [ ]:
importlib.reload(act)

act.plot_awd(clip_dat,plots,max_act=500,show=False,comments=gc,fn_pref=os.path.join(out_dir,sub_long+'_M+logs'),plot_type='single',debug=True)


In [ ]:
importlib.reload(act)

act.plot_awd(clip_dat,plots,max_act=250,show=False,comments=gc,fn_pref=os.path.join(out_dir,sub_long+'_M+logs_zoom'),plot_type='single',debug=True)
